## Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import math
from itertools import combinations
import time

In [2]:
adult = pd.read_excel('adult.xlsx')
adult.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,Annual Salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
# bucketing age and hours per week into buckets of width 5
width = 5

bins = math.ceil((max(adult["age"]) - min(adult["age"]))/width)
adult["age_bucket"] = (np.floor(adult["age"]/width)*width).astype(str) + " - " + ((np.floor(adult["age"]/width)+1)*width).astype(str) + " years"

bins = math.ceil((max(adult["hours_per_week"]) - min(adult["hours_per_week"]))/width)
adult["hours_per_week_bucket"] = (np.floor(adult["hours_per_week"]/width)*width).astype(str) + " - " + ((np.floor(adult["hours_per_week"]/width)+1)*width).astype(str) + " hours/week"

adult.loc[adult["workclass"] == ' ?', "workclass"] = 'Unknown'
adult.loc[adult["occupation"] == ' ?', "occupation"] = 'Unknown'
adult.loc[adult["native_country"] == ' ?', "native_country"] = 'Unknown'

remove_list = []

# remove fnlwgt because too many unique values
# remove capital_gain, capital_loss because around 90% values are 0
    
for x in adult.columns:
    if len(adult[x].value_counts()) > 50:
        remove_list.append(x)
    
# remove education_num because it is same as education
remove_list.append('education_num')

final_col_list = [x for x in adult.columns if x not in remove_list]

adult['List of items'] = adult['workclass']
for i in final_col_list:
    if i!= 'workclass':
        adult['List of items'] = adult['List of items'].astype(str) + ',' + adult[i].astype(str)

## Apriori Algorithm


In [4]:
min_sup = 0.5

In [5]:
sample_data = adult[['List of items']]
min_sup = math.floor(min_sup * len(sample_data))
# min_sup = 15081.5
min_sup

In [17]:
unique_list = []
for i in sample_data['List of items']:
    unique_list.append((',').join(sorted(list(set(i.split(','))))))
    
sample_data['List of items'] = unique_list

<ipython-input-17-d54439ce0b96>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_data['List of items'] = unique_list


In [19]:
sample_data

,List of items
0,"<=50K, Adm-clerical, Bachelors, Male, Never-m..."
1,"<=50K, Bachelors, Exec-managerial, Husband, M..."
2,"<=50K, Divorced, HS-grad, Handlers-cleaners, ..."
3,"11th, <=50K, Black, Handlers-cleaners, Husban..."
4,"<=50K, Bachelors, Black, Cuba, Female, Marrie..."
...,...
32556,"<=50K, Assoc-acdm, Female, Married-civ-spouse..."
32557,">50K, HS-grad, Husband, Machine-op-inspct, Ma..."
32558,"<=50K, Adm-clerical, Female, HS-grad, Private..."
32559,"<=50K, Adm-clerical, HS-grad, Male, Never-mar..."


In [20]:
def generate_next_level_itemsets(L, level):
    dict_count = {}
    for i in range(0, L.shape[0]):
        for j in range(i+1,L.shape[0]):
            if len(set(L['Itemset'][i]) ^ set(L['Itemset'][j])) == 2:
                dict_count[(',').join(sorted(list(set(L['Itemset'][i]+L['Itemset'][j]))))] = 0
                
    # checking the Apriori property
    for k in list(dict_count.keys()):
        for i in list(combinations(k.split(','), level - 1)):
            if list(i) not in list(L['Itemset']):
                del dict_count[k]
                break
    return dict_count
    
def count_transactions(dict_count, sample_data):
    # to count the no. of transactions for each items
    for i in sample_data['List of items'].str.split(','):
        for k in dict_count.keys():
            if((set(k.split(',')) & set(i)) == set(k.split(','))):
                dict_count[k]+=1
    
    return dict_count

def generate_C_and_L(dict_count, min_sup):
    list_keys = []
    list_values = []

    for key,value in dict_count.items():
        list_keys.append(key.split(','))
        list_values.append(value)

    C = pd.DataFrame({'Itemset':list_keys, 'Support_count':list_values})

    L = C[C['Support_count'] >= min_sup].reset_index(drop = True)
    
    return C, L


In [21]:
def apriori(sample_data, min_sup):
    
    print("Minimum Support: ", min_sup)
    
    level = 1
    
    dict_final = {}
    
    dict_count = {}
    # to add the items
    for i in sample_data['List of items'].str.split(','):
        for j in i:
            dict_count[j] = 0

    # to count the no. of transactions for each items
    for i in sample_data['List of items'].str.split(','):
        for k in dict_count.keys():
            if k in i:
                dict_count[k]+=1

    list_keys = []
    list_values = []

    for key,value in dict_count.items():
        list_keys.append(key.split(','))
        list_values.append(value)

    for k in list(dict_count.keys()):
            if dict_count[k] < min_sup:
                del dict_count[k]
        
    dict_final.update(dict_count)
    
    C = pd.DataFrame({'Itemset':list_keys, 'Support_count':list_values})

    L = C[C['Support_count'] >= min_sup].reset_index(drop = True)

    
    while(len(L)>0):
        print("C"+str(level) + ":", C)
        print("L"+str(level) + ":", L)
        level+=1
        dict_count = generate_next_level_itemsets(L, level)
        dict_count = count_transactions(dict_count, sample_data)
        C, L = generate_C_and_L(dict_count, min_sup)
        
        for k in list(dict_count.keys()):
            if dict_count[k] < min_sup:
                del dict_count[k]
                
        dict_final.update(dict_count)
        
    return dict_final

In [22]:
apriori_time = time.time()
dict_final_apriori = apriori(sample_data, min_sup)
apriori_time = time.time() - apriori_time

Minimum Support:  15081.5
C1:                       Itemset  Support_count
0                    [ <=50K]          24720
1             [ Adm-clerical]           3770
2                [ Bachelors]           5355
3                     [ Male]          21790
4            [ Never-married]          10683
..                        ...            ...
133  [85.0 - 90.0 hours/week]             20
134                [ Ireland]             24
135           [ Never-worked]              7
136                [ Hungary]             13
137     [ Holand-Netherlands]              1

[138 rows x 2 columns]
L1:                     Itemset  Support_count
0                  [ <=50K]          24720
1                   [ Male]          21790
2          [ United-States]          29170
3                  [ White]          27816
4  [40.0 - 45.0 hours/week]          15835
5                [ Private]          22696
C2:                                      Itemset  Support_count
0                            [ <=50K,

In [15]:
len(dict_final_apriori)

16

In [14]:
apriori_time

3.1210203170776367

## Improved Apriori (adding Transaction reduction)

In [190]:
def transaction_reduction(sample_data, L):
    itemset_needed = []
    for i in range(0, len(sample_data['List of items'])):
        needed = False
        for j in range(0, L.shape[0]):
            if set(L['Itemset'][j]) & set((sample_data['List of items'][i]).split(',')) == set(L['Itemset'][j]):
                needed = True
                break

        itemset_needed.append(needed)
    
    sample_data['itemset_needed'] = itemset_needed
    
    sample_data = sample_data[sample_data['itemset_needed'] == True].reset_index(drop = True)
    
    print(len(sample_data))
    
    return sample_data
    

def generate_next_level_itemsets(L, level):
    dict_count = {}
    for i in range(0, L.shape[0]):
        for j in range(i+1,L.shape[0]):
            if len(set(L['Itemset'][i]) ^ set(L['Itemset'][j])) == 2:
                dict_count[(',').join(sorted(list(set(L['Itemset'][i]+L['Itemset'][j]))))] = 0

    # checking the Apriori property
    for k in list(dict_count.keys()):
        for i in list(combinations(k.split(','), level - 1)):
            if list(i) not in list(L['Itemset']):
                del dict_count[k]
                break
    return dict_count
    
def count_transactions(dict_count, sample_data):
    # to count the no. of transactions for each items
    for i in sample_data['List of items'].str.split(','):
        for k in dict_count.keys():
            if((set(k.split(',')) & set(i)) == set(k.split(','))):
                dict_count[k]+=1
    
    return dict_count

def generate_C_and_L(dict_count, min_sup):
    list_keys = []
    list_values = []

    for key,value in dict_count.items():
        list_keys.append(key.split(','))
        list_values.append(value)

    C = pd.DataFrame({'Itemset':list_keys, 'Support_count':list_values})

    L = C[C['Support_count'] >= min_sup].reset_index(drop = True)
    
    return C, L


In [191]:
def improved_apriori(sample_data, min_sup):
    
    print("Minimum Support: ", min_sup)
    
    level = 1
    
    dict_final = {}
    
    dict_count = {}
    # to add the items
    for i in sample_data['List of items'].str.split(','):
        for j in i:
            dict_count[j] = 0

    # to count the no. of transactions for each items
    for i in sample_data['List of items'].str.split(','):
        for k in dict_count.keys():
            if k in i:
                dict_count[k]+=1

    list_keys = []
    list_values = []

    for key,value in dict_count.items():
        list_keys.append(key.split(','))
        list_values.append(value)

    for k in list(dict_count.keys()):
            if dict_count[k] < min_sup:
                del dict_count[k]
        
    dict_final.update(dict_count)
    
    C = pd.DataFrame({'Itemset':list_keys, 'Support_count':list_values})

    L = C[C['Support_count'] >= min_sup].reset_index(drop = True)

    
    while(len(L)>0):
        print("C"+str(level) + ":", C)
        print("L"+str(level) + ":", L)
        level+=1
        sample_data = transaction_reduction(sample_data, L)
        dict_count = generate_next_level_itemsets(L, level)
        dict_count = count_transactions(dict_count, sample_data)
        C, L = generate_C_and_L(dict_count, min_sup)
        
        for k in list(dict_count.keys()):
            if dict_count[k] < min_sup:
                del dict_count[k]
    
        dict_final.update(dict_count)
        
    return dict_final

In [192]:
improved_apriori_time = time.time()
dict_final_improved_apriori = improved_apriori(sample_data, min_sup)
improved_apriori_time = time.time() - improved_apriori_time

Minimum Support:  16280
C1:                       Itemset  Support_count
0                    [ <=50K]          24720
1             [ Adm-clerical]           3770
2                [ Bachelors]           5355
3                     [ Male]          21790
4            [ Never-married]          10683
..                        ...            ...
133  [85.0 - 90.0 hours/week]             20
134                [ Ireland]             24
135           [ Never-worked]              7
136                [ Hungary]             13
137     [ Holand-Netherlands]              1

[138 rows x 2 columns]
L1:             Itemset  Support_count
0          [ <=50K]          24720
1           [ Male]          21790
2  [ United-States]          29170
3          [ White]          27816
4        [ Private]          22696


<ipython-input-190-9534b1e27394>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_data['itemset_needed'] = itemset_needed


32544
C2:                       Itemset  Support_count
0             [ <=50K,  Male]          15128
1    [ <=50K,  United-States]          21999
2            [ <=50K,  White]          20699
3          [ <=50K,  Private]          17733
4     [ Male,  United-States]          19488
5             [ Male,  White]          19174
6           [ Male,  Private]          14944
7    [ United-States,  White]          25621
8  [ Private,  United-States]          20135
9          [ Private,  White]          19404
L2:                       Itemset  Support_count
0    [ <=50K,  United-States]          21999
1            [ <=50K,  White]          20699
2          [ <=50K,  Private]          17733
3     [ Male,  United-States]          19488
4             [ Male,  White]          19174
5    [ United-States,  White]          25621
6  [ Private,  United-States]          20135
7          [ Private,  White]          19404
32011
C3:                               Itemset  Support_count
0    [ <=50K,  United-S

## Another Improvement try (sampling)

In [193]:
def generate_next_level_itemsets(L, level):
    dict_count = {}
    for i in range(0, L.shape[0]):
        for j in range(i+1,L.shape[0]):
            if len(set(L['Itemset'][i]) ^ set(L['Itemset'][j])) == 2:
                dict_count[(',').join(sorted(list(set(L['Itemset'][i]+L['Itemset'][j]))))] = 0

    # checking the Apriori property
    for k in list(dict_count.keys()):
        for i in list(combinations(k.split(','), level - 1)):
            if list(i) not in list(L['Itemset']):
                del dict_count[k]
                break
    return dict_count
    
def count_transactions(dict_count, sample_data):
    # to count the no. of transactions for each items
    for i in sample_data['List of items'].str.split(','):
        for k in dict_count.keys():
            if((set(k.split(',')) & set(i)) == set(k.split(','))):
                dict_count[k]+=1
    
    return dict_count

def generate_C_and_L(dict_count, min_sup):
    list_keys = []
    list_values = []

    for key,value in dict_count.items():
        list_keys.append(key.split(','))
        list_values.append(value)

    C = pd.DataFrame({'Itemset':list_keys, 'Support_count':list_values})

    L = C[C['Support_count'] >= min_sup].reset_index(drop = True)
    
    return C, L


In [194]:
def apriori(sample_data, min_sup):
    
    print("Minimum Support: ", min_sup)
    
    level = 1
    
    dict_final = {}
    
    dict_count = {}
    # to add the items
    for i in sample_data['List of items'].str.split(','):
        for j in i:
            dict_count[j] = 0

    # to count the no. of transactions for each items
    for i in sample_data['List of items'].str.split(','):
        for k in dict_count.keys():
            if k in i:
                dict_count[k]+=1

    list_keys = []
    list_values = []

    for key,value in dict_count.items():
        list_keys.append(key.split(','))
        list_values.append(value)

    for k in list(dict_count.keys()):
            if dict_count[k] < min_sup:
                del dict_count[k]
        
    dict_final.update(dict_count)
    
    C = pd.DataFrame({'Itemset':list_keys, 'Support_count':list_values})

    L = C[C['Support_count'] >= min_sup].reset_index(drop = True)

    
    while(len(L)>0):
        print("C"+str(level) + ":", C)
        print("L"+str(level) + ":", L)
        level+=1
        dict_count = generate_next_level_itemsets(L, level)
        dict_count = count_transactions(dict_count, sample_data)
        C, L = generate_C_and_L(dict_count, min_sup)
        
        for k in list(dict_count.keys()):
            if dict_count[k] < min_sup:
                del dict_count[k]
                
        dict_final.update(dict_count)
        
    return dict_final

In [195]:
sample_size = 0.2

indexes = list(np.random.choice(32561, math.floor(sample_size * 32561) , replace=False))

second_improvement_apriori_time = time.time()

sample_data_new = sample_data.iloc[indexes].reset_index(drop = True)

dict_final_second_improvement_apriori = apriori(sample_data_new, min_sup)
second_improvement_apriori_time = time.time() - second_improvement_apriori_time

Minimum Support:  16280


In [196]:
apriori_time

1.9971296787261963

In [197]:
improved_apriori_time

4.341088056564331

In [198]:
second_improvement_apriori_time

0.2765533924102783

## FP - growth

In [199]:
# Trie
class TrieNode:
    def __init__(self, item):
        self.item = item
        self.children = {}
        self.end = False
        
class Trie:
    def __init__(self):
        self.root = TrieNode("")
        
    def insert(self, itemset):
        node = self.root
        for item in itemset:
            if item in node.children:
                node.children[item][1] += 1
                node = node.children[item][0]
            else:
                new = TrieNode(item)
                
                node.children[item] = [new, 1]
                node = new
        
        node.end = True

def FP_growth(sample_data, min_sup):
    fpgrowth_time = time.time()
    
    print("Minimum Support: ", min_sup)
    
    dict_count = {}
    # to add the items
    for i in sample_data['List of items'].str.split(','):
        for j in i:
            dict_count[j] = 0
        
    
    # to count the no. of transactions for each items
    for i in sample_data['List of items'].str.split(','):
        for k in dict_count.keys():
            if k in i:
                dict_count[k]+=1

    list_keys = []
    list_values = []

    for key,value in dict_count.items():
        list_keys.append(key.split(','))
        list_values.append(value)
    
    C = pd.DataFrame({'Itemset':list_keys, 'Support_count':list_values})

    L = C[C['Support_count'] >= min_sup].reset_index(drop = True)
    
    L_sorted = L.sort_values(by = ['Support_count'], ascending= False).reset_index(drop = True)

    sorting_order = list([x[0] for x in L_sorted['Itemset']])

    sample_data['Itemset'] = sample_data['List of items'].str.split(',')
    
    fpgrowth_time = time.time() - fpgrowth_time
    # ordering all set according to the sorting order
    
    for i in range(0,len(sample_data['Itemset'])):

        sample_data['Itemset'][i] = [ x for x in sorting_order if x in sample_data['Itemset'][i]]
            
    fpgrowth_time = time.time() - fpgrowth_time
    
    fp = Trie()
    for i in range(0,len(sample_data['Itemset'])):
        
        fp.insert(sample_data['Itemset'][i])

    current_node = fp.root

    def traverse(current_node, parent_item, item_to_search, val):
        if current_node.item == item_to_search:
            conditional_probs[','.join([x for x in parent_item.split(',') if x!=''])] = val

        for k in current_node.children.values():
            traverse(k[0], parent_item + ',' + current_node.item, item_to_search, k[1])
    
    all_conditional_probs = []
    for i in list(L_sorted['Itemset'])[::-1]:
        conditional_probs = {}
        traverse(current_node, "", i[0], 0)
        if '' in conditional_probs.keys():
            del conditional_probs['']
        all_conditional_probs.append(conditional_probs)

    Conditional_Pattern_Base_df = pd.DataFrame({'Item':list(L_sorted['Itemset'])[::-1],'Conditional Pattern Base':all_conditional_probs})
    Conditional_Pattern_Base_df = Conditional_Pattern_Base_df[Conditional_Pattern_Base_df['Conditional Pattern Base']!={}].reset_index(drop = True)
    
    final_results_fp = []

    for n in range(0,len(Conditional_Pattern_Base_df['Conditional Pattern Base'])):
        dict_results = {}
        Conditional_Pattern_Base_df['Conditional Pattern Base'][n]
        list_needed = []
        for i in sorting_order:
            if i == Conditional_Pattern_Base_df['Item'][n][0]:
                break 

            list_needed.append(i)

        final_combinations = []
        for j in range(1,len(list_needed)+1):
            final_combinations+= combinations(list_needed, j)

        for k in range(len(final_combinations)):
            final_combinations[k] = list(final_combinations[k])
            dict_results[(',').join(list(final_combinations[k]))] = 0

        for k,v in Conditional_Pattern_Base_df['Conditional Pattern Base'][n].items():
            for l in list(dict_results.keys()):
                if set(k.split(',')) & set(l.split(',')) == set(l.split(',')):
                    dict_results[l]+=v

        for k in list(dict_results.keys()):
            if dict_results[k] < min_sup:
                del dict_results[k]

        dict_final_results = {}
        for k in list(dict_results.keys()):
            dict_final_results[k + ',' + Conditional_Pattern_Base_df['Item'][n][0]] = dict_results[k]


        final_results_fp.append(dict_final_results)
                
    fpgrowth_time = time.time() - fpgrowth_time
        
    return Conditional_Pattern_Base_df, final_results_fp, dict_count, fpgrowth_time


In [200]:
Conditional_Pattern_Base_df, final_results_fp, dict_count, fpgrowth_time = FP_growth(sample_data, min_sup)

for k in list(dict_count.keys()):
    if dict_count[k] < min_sup:
        del dict_count[k]
    
dict_final_fp_growth = {}

for i in final_results_fp:
    dict_final_fp_growth.update(i)
    
dict_final_fp_growth.update(dict_count)



Minimum Support:  16280


<ipython-input-199-53423c1f0f6e>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_data['Itemset'] = sample_data['List of items'].str.split(',')
<ipython-input-199-53423c1f0f6e>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_data['Itemset'][i] = [ x for x in sorting_order if x in sample_data['Itemset'][i]]
<ipython-input-200-f0cc3c25da26>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [201]:
Conditional_Pattern_Base_df

,Item,Conditional Pattern Base
0,[ Male],"{' United-States, White, <=50K': 3632, ' Unite..."
1,[ Private],"{' United-States, White, <=50K': 13452, ' Unit..."
2,[ <=50K],"{' United-States, White': 18917, ' United-Stat..."
3,[ White],{' United-States': 25621}


## Final answers

In [202]:
itemsets = []
support_count = []

for k,v in dict_final_apriori.items():
    itemsets.append(k.split(','))
    support_count.append(v)
    
apriori_result_df = pd.DataFrame({'Itemsets':itemsets, 'Support Count':support_count})

apriori_result_df

,Itemsets,Support Count
0,[ <=50K],24720
1,[ Male],21790
2,[ United-States],29170
3,[ White],27816
4,[ Private],22696
5,"[ <=50K, United-States]",21999
6,"[ <=50K, White]",20699
7,"[ <=50K, Private]",17733
8,"[ Male, United-States]",19488
9,"[ Male, White]",19174


In [203]:
print(dict_final_apriori)

print("--------------------")

print(len(dict_final_apriori))

{' <=50K': 24720, ' Male': 21790, ' United-States': 29170, ' White': 27816, ' Private': 22696, ' <=50K, United-States': 21999, ' <=50K, White': 20699, ' <=50K, Private': 17733, ' Male, United-States': 19488, ' Male, White': 19174, ' United-States, White': 25621, ' Private, United-States': 20135, ' Private, White': 19404, ' <=50K, United-States, White': 18917, ' Male, United-States, White': 17653, ' Private, United-States, White': 17728}
--------------------
16


In [204]:
itemsets = []
support_count = []

for k,v in dict_final_fp_growth.items():
    itemsets.append(k.split(','))
    support_count.append(v)
    
fpgrowth_result_df = pd.DataFrame({'Itemsets':itemsets, 'Support Count':support_count})

fpgrowth_result_df

,Itemsets,Support Count
0,"[ United-States, Male]",19488
1,"[ White, Male]",19174
2,"[ United-States, White, Male]",17653
3,"[ United-States, Private]",20135
4,"[ White, Private]",19404
5,"[ <=50K, Private]",17733
6,"[ United-States, White, Private]",17728
7,"[ United-States, <=50K]",21999
8,"[ White, <=50K]",20699
9,"[ United-States, White, <=50K]",18917


In [205]:
print(dict_final_fp_growth)

print("--------------------")

print(len(dict_final_fp_growth))

{' United-States, Male': 19488, ' White, Male': 19174, ' United-States, White, Male': 17653, ' United-States, Private': 20135, ' White, Private': 19404, ' <=50K, Private': 17733, ' United-States, White, Private': 17728, ' United-States, <=50K': 21999, ' White, <=50K': 20699, ' United-States, White, <=50K': 18917, ' United-States, White': 25621, ' <=50K': 24720, ' Male': 21790, ' United-States': 29170, ' White': 27816, ' Private': 22696}
--------------------
16


## Time Comparisons

In [206]:
apriori_time

1.9971296787261963

In [207]:
fpgrowth_time

1.2573974132537842